## Sprint 4: Build a model for forecasting the number of customers of the merchant

### Step 1:  Data generating

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('../../project-2-group-buy-now-pay-later-industry-project-4/data/tables/df_new_week8.csv')
print(df.columns)
df['dollar_value'] = df['dollar_value'].fillna(0)
# find number of different consumer_id by group by merchant_abn
consumer_count = df.groupby('merchant_abn')['consumer_id'].nunique().reset_index(name='count')
# Group other variables are according to merchant_abn
def mode_group(group):
    modes = {}
    for col in group.columns:
        if col == 'merchant_abn':
            continue
        try:
            modes[col] = pd.Series.mode(group[col])[0]
        except IndexError:
            modes[col] = np.nan
    return pd.Series(modes)
consumer_dsum = df.groupby('merchant_abn')['dollar_value'].sum().reset_index(name='dollar_sum')
print(consumer_dsum.head(1))
consumer_tmp1 = df.groupby('merchant_abn')[['fraud_probability_x','fraud_probability_y','cpi']].mean().reset_index()
print(consumer_tmp1.head(1))
consumer_tmp2 = df.groupby('merchant_abn')[['state','postcode','gender','is_fraud']].apply(mode_group).reset_index()
print(consumer_tmp2.head(1))
# merge them together 
df_4 = pd.merge(consumer_count,consumer_dsum)
df_4 = pd.merge(df_4,consumer_tmp1)
df_4 = pd.merge(df_4,consumer_tmp2)
df_4.head(2)
df_4.to_csv('../../project-2-group-buy-now-pay-later-industry-project-4/data/tables/df_new_week9.csv')


Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'order_datetime',
       'user_id', 'merchant_abn', 'dollar_value', 'fraud_probability_x',
       'fraud_probability_y', 'consumer_id', 'name', 'address', 'state',
       'postcode', 'gender', 'year', 'month', 'cpi', 'is_fraud'],
      dtype='object')
   merchant_abn    dollar_sum
0   11149063370  164023.03278
   merchant_abn  fraud_probability_x  fraud_probability_y         cpi
0   11149063370            51.588733            11.721278  108.877453
   merchant_abn state  postcode  gender  is_fraud
0   11149063370   NSW      4311  Female      True


### step2: Build a model to predict number

In [2]:
# Build a linear regression model to predict the number of customers
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
ohe = OneHotEncoder()
scaler = StandardScaler()
X = df_4[['dollar_sum','fraud_probability_x','fraud_probability_y','cpi']]
# standardizing the data

X = pd.DataFrame(scaler.fit_transform(X),columns=['dollar_sum','fraud_probability_x','fraud_probability_y','cpi'])
X_transformed = ohe.fit_transform(df_4[['state','gender','postcode']]).toarray()
print(ohe.categories_)
c = [str(j) for i in ohe.categories_ for j in i]
X_transformed = pd.DataFrame(X_transformed, columns=c)
for name in c:
    X[name] = list(X_transformed[name])
y = df_4['count']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)
print(X_train.head(2))
lm = LinearRegression()
lm.fit(X_train,y_train)
print(lm.coef_)


[array(['NSW', 'QLD', 'SA', 'VIC', 'WA'], dtype=object), array(['Female', 'Male'], dtype=object), array([ 800,  850,  881, 1026, 1028, 1132, 1166, 1177, 1229, 1470, 1480,
       1835, 2106, 2430, 2582, 2757, 2867, 2869, 3387, 3521, 4311, 4825,
       5132, 6718, 7182])]
    dollar_sum  fraud_probability_x  fraud_probability_y       cpi  NSW  QLD  \
19   -0.868763            -1.890365             0.014704 -0.166445  0.0  0.0   
25   -0.406762            -0.299954            -0.086352 -0.166445  1.0  0.0   

     SA  VIC   WA  Female  ...  2757  2867  2869  3387  3521  4311  4825  \
19  1.0  0.0  0.0     0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
25  0.0  0.0  0.0     1.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

    5132  6718  7182  
19   1.0   0.0   0.0  
25   0.0   0.0   0.0  

[2 rows x 36 columns]
[-9.84812232e-12  5.05304581e-13 -1.44033260e+02  5.87904897e+00
  5.98520578e+01  1.67423467e+02 -6.31304108e+01 -6.62053537e+01
 -9.62840149e+01 -2.41165148e+01  2.2

In [3]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
y_tpred = lm.predict(X_train)
y_pred = lm.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
r2t = r2_score(y_train, y_tpred)
print(f"Mean Squared Error: {mse}")
print(f"train R-squared: {r2t}")
print(f"test R-squared: {r2}")

Mean Absolute Error: 62.76977377010564
Mean Squared Error: 7611.28189678359
train R-squared: 1.0
test R-squared: 0.10934724123301687


R2=1 on the training set indicates a good fit of the model. From the parameter results of linear regression, it can be found that dollar_sum has the greatest impact on the consumer count.